In [1]:
def stock():
    costAverage = 0
    totalCost = 0
    totalStocks = 0
    #numberOfPrices = int(input("How many prices did you buy the stock at?"))
    price = list(map(float, input("What are the prices you paid for the stock?").split()))
    stockNumber = list(map(float, input("What are the corresponding amount of stocks for each price?").split()))
    for i in range(len(stockNumber)):
        totalStocks = totalStocks + stockNumber[0+i]
    i=0
    print("You bought the stock",len(stockNumber), "times")
    print("Total amount of stocks purchased:", totalStocks)
    print("Price list:", price)
    print("Volume list:",stockNumber)
    for i in range(len(stockNumber)):
        totalCost = totalCost + ((stockNumber[i])* price[i])
    #print(totalCost)
    costAverage = (totalCost/totalStocks)
    print("Your cost average is:", costAverage)
    #print(costAverage)
    return(price, stockNumber, totalCost, totalStocks)


In [2]:
stock()

You bought the stock 14 times
Total amount of stocks purchased: 900.0
Price list: [20.0, 15.5, 14.5, 13.5, 13.5, 14.0, 13.0, 13.0, 13.5, 14.0, 15.5, 16.5, 18.0, 18.5]
Volume list: [200.0, 100.0, -100.0, -100.0, 100.0, 100.0, -100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0]
Your cost average is: 16.77777777777778


([20.0,
  15.5,
  14.5,
  13.5,
  13.5,
  14.0,
  13.0,
  13.0,
  13.5,
  14.0,
  15.5,
  16.5,
  18.0,
  18.5],
 [200.0,
  100.0,
  -100.0,
  -100.0,
  100.0,
  100.0,
  -100.0,
  100.0,
  100.0,
  100.0,
  100.0,
  100.0,
  100.0,
  100.0],
 15100.0,
 900.0)

In [3]:
def get_stock_price(stockTicker):
    import yfinance as yf
    typeOfStock = yf.Ticker(stockTicker)
    currentPrice = typeOfStock.history(period = '1d')['Close'].iloc[-1]  
    print(currentPrice)
    intStockPrice = int(currentPrice*100)
    #print(intStockPrice)
    currentStockPrice = intStockPrice/100
    return(currentStockPrice)
    

In [4]:
def DollarCostAveraging(price, stockNumber, totalCost, totalStocks):
    computation = 0
    batch = 0
    filtered_totalCost = 0
    filtered_totalStocks = 0
    
#getting actual stock price
    print()
    stockTicker = input("What is the stock symbol of the stock you bought? (make sure it is in all caps)")
    currentStockPrice = get_stock_price(stockTicker)
    print(currentStockPrice)
    print("Currrent price of", stockTicker,"or a $0.01 lower:", currentStockPrice)
    print()
#manual stock price input
    #currentStockPrice = float(input("What is the current price of the stock?"))
    #print(currentStockPrice)

#getting the most recent at the money price
    atm_call_strike, atm_call_strike_next_higher, atm_put_strike, expiration_date = get_atm_strikes(stockTicker)
    print("ATM Call Strike Price:", atm_call_strike)
    print("ATM Call Strike Price (Next Higher):", atm_call_strike_next_higher)
    print("ATM Put Strike Price:", atm_put_strike)
    print("Most Recent Expiration Date:", expiration_date)
    print()

#computing highest price and filtering a list with it and all prices lower than currentStockPrice
    #data_dict = dict(zip(price, stockNumber))

    # Create a dictionary with prices and corresponding volumes
    data_dict = {}
    for p, num in zip(price, stockNumber):
        if p in data_dict:
        # If the price already exists, accumulate the volume
            data_dict[p] += num
        else:
        # If the price is new, add it to the dictionary
            data_dict[p] = num

# Rest of your code...


    max_price = max(data_dict.keys())
    max_volume = data_dict[max_price]
    print("Highest price:", max_price, "with a volume of:", max_volume)
    filtered_price = [price for price in data_dict.keys() if price < atm_call_strike_next_higher+1]
    print(filtered_price)
    filtered_price.append(max_price)
    filtered_stockNumber = [data_dict[price] for price in filtered_price]
    i= 0
    for i in range(len(filtered_stockNumber)):
        filtered_totalStocks= filtered_totalStocks + filtered_stockNumber[0+i]
    i= 0
    print("Filtered prices below (", atm_call_strike_next_higher+1, ") + highest price:", filtered_price)
    print("Filtered coressponding volume + highest price's volume:", filtered_stockNumber)
    print("Total stocks in filtered list:", filtered_totalStocks)
    print()
    
    print("Computations for:", max_price)
    for i in range(len(filtered_price)):
        filtered_totalCost = filtered_totalCost + ((filtered_stockNumber[i])* filtered_price[i])
    #print(filtered_totalCost)

    #shows breakeven cost
    breakEven = filtered_totalCost/filtered_totalStocks
    print("Your breakeven price with your filtered list consisting of:", filtered_price, "is:", breakEven)

    while computation == 0 or computation > (atm_call_strike_next_higher+1):
        computation = ((filtered_totalCost + (atm_put_strike-1)*(100*batch))/(filtered_totalStocks + (100*batch)))
        #computes the future price you need to sell the modified list at with incremental amounts of stock options needed to be bought at current price until the future price become close to the rounded up current price
        #print("Buy", batch*100 , "stocks at current price, and sell it at:",computation)
        batch = batch+1
    stockToBuy = ((batch-1))
    print(computation)
    print("You will need to sell", stockToBuy, "puts (buy", stockToBuy*100,  "stocks) to make a profit of", ((atm_call_strike_next_higher+1)-computation)*filtered_totalStocks)

    
    #makes a list of the rest of the prices higher than market value
    higherPrices = [price for price in data_dict.keys() if price > currentStockPrice]
    higherPrices = sorted(higherPrices)
    print()
    print(higherPrices[:-1])
    print()

#Does the dollar cost averaging for the remaining prices above market value
    for count in range(len(higherPrices)-1):
        removingLastprice = higherPrices.pop()
        filtered_price.pop()
        
        selectedHighPrice = higherPrices[-1]
        print("Computations for:",selectedHighPrice)
        filtered_price.append(selectedHighPrice)
        print(filtered_price)
        filtered_stockNumber = [data_dict[price] for price in filtered_price]
        print(filtered_stockNumber)
        
        filtered_totalCost  = 0
        i= 0
        filtered_totalStocks = 0
    #calculates total stocks and total costs
        for i in range(len(filtered_stockNumber)):
            filtered_totalStocks= filtered_totalStocks + filtered_stockNumber[0+i]
        i= 0
        for i in range(len(filtered_price)):
            filtered_totalCost = filtered_totalCost + ((filtered_stockNumber[i])* filtered_price[i])
            #print(filtered_totalCost)

        #shows breakeven cost for other high prices
        breakEven = filtered_totalCost/filtered_totalStocks
        print("Your breakeven price with your filtered list consisting of:", filtered_price, "is:", breakEven)

        #computation = currentStockPrice+2
        batch = 0
        computation = ((filtered_totalCost + currentStockPrice*(100*batch))/(filtered_totalStocks + (100*batch)))
        over = 0
        while computation == 0 or computation > (atm_call_strike_next_higher+1):
            computation = ((filtered_totalCost + (atm_put_strike-1)*(100*batch))/(filtered_totalStocks + (100*batch)))
            #computes the future price you need to sell the modified list at with incremental amounts of stock options needed to be bought at current price until the future price become close to the rounded up current price
            #print("Buy", batch*100 , "stocks at current price, and sell it at:",computation)
            batch = batch+1
            over = 1
        stockToBuy = batch
        if over == 1:
            stockToBuy = ((batch-1))
        print(computation)
        print("You will need to sell", stockToBuy, "puts (buy", stockToBuy*100,  "stocks) to make a profit of", ((atm_call_strike_next_higher+1)-computation)*filtered_totalStocks)
        print()
    
        


       
        





In [5]:
import yfinance as yf
import pandas as pd

def get_atm_strikes(stock_symbol):
    # Get the stock object
    stock_obj = yf.Ticker(stock_symbol)

    # Get the option chain for the most recent expiration date
    expiration_dates = stock_obj.options
    latest_expiration_date = expiration_dates[0]
    option_chain = stock_obj.option_chain(latest_expiration_date)

    # Get the stock's current price
    current_price = stock_obj.history(period='1d')['Close'].iloc[-1]

    # Access the options data
    options = pd.concat([option_chain.calls, option_chain.puts]).reset_index(drop=True)

    # Find the ATM call and put options
    atm_call_strike_idx = (options['strike'] - current_price).abs().idxmin()
    atm_call_strike = options.loc[atm_call_strike_idx]['strike']

    # Find the next higher strike for the ATM call
    next_higher_call_strike_idx = options.loc[options['strike'] > atm_call_strike].index.min()
    atm_call_strike_next_higher = options.loc[next_higher_call_strike_idx]['strike']

    # Find the ATM put option
    atm_put_strike = options.iloc[(options['strike'] - current_price).abs().idxmin()]['strike']

    return atm_call_strike, atm_call_strike_next_higher, atm_put_strike, latest_expiration_date

# Replace 'AAPL' with the stock symbol
stock_symbol = 'PINS'
atm_call_strike, atm_call_strike_next_higher, atm_put_strike, expiration_date = get_atm_strikes(stock_symbol)

# Now you have the adjusted strike prices and the most recent expiration date as variables
print("ATM Call Strike Price:", atm_call_strike)
print("ATM Call Strike Price (Next Higher):", atm_call_strike_next_higher)
print("ATM Put Strike Price:", atm_put_strike)
print("Most Recent Expiration Date:", expiration_date)


ATM Call Strike Price: 34.0
ATM Call Strike Price (Next Higher): 34.5
ATM Put Strike Price: 34.0
Most Recent Expiration Date: 2023-12-01


In [6]:
#pip install yfinance
import yfinance as yf
price, stockNumber, totalCost, totalStocks = stock()
#atm_call_strike, atm_put_strike, latest_expiration_date = get_atm_strikes("PINS") 
DollarCostAveraging(price, stockNumber, totalCost, totalStocks)

You bought the stock 14 times
Total amount of stocks purchased: 900.0
Price list: [20.0, 15.5, 14.5, 13.5, 13.5, 14.0, 13.0, 13.0, 13.5, 14.0, 15.5, 16.5, 18.0, 18.5]
Volume list: [200.0, 100.0, -100.0, -100.0, 100.0, 100.0, -100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0]
Your cost average is: 16.77777777777778

14.979999542236328
14.97
Currrent price of VALE or a $0.01 lower: 14.97

ATM Call Strike Price: 15.0
ATM Call Strike Price (Next Higher): 15.5
ATM Put Strike Price: 15.0
Most Recent Expiration Date: 2023-12-01

Highest price: 20.0 with a volume of: 200.0
[15.5, 14.5, 13.5, 14.0, 13.0]
Filtered prices below ( 16.5 ) + highest price: [15.5, 14.5, 13.5, 14.0, 13.0, 20.0]
Filtered coressponding volume + highest price's volume: [200.0, -100.0, 100.0, 200.0, 0.0, 200.0]
Total stocks in filtered list: 600.0

Computations for: 20.0
Your breakeven price with your filtered list consisting of: [15.5, 14.5, 13.5, 14.0, 13.0, 20.0] is: 16.333333333333332
16.333333333333332
You will